# 机器博弈算法

我们的任务是创建一个可以玩并赢得 Impact Isolation 游戏的 AI。

# 目录表

0. [关于游戏](#关于游戏)
1. [重要文件](#重要文件)
2. [任务](#任务)
3. [导入外部模块](#导入外部模块)
4. [评估函数](#评估函数)
5. [游戏代理](#游戏代理)
6. [Minimax](#Minimax)
7. [AlphaBeta](#AlphaBeta)
8. [提高性能](#提高性能)
9. [代理大战](#代理大战)

---
<!-- Hi there! -->

# 关于游戏

Impact Isolation 的规则是原 Isolation 游戏的变体。在游戏的原始形式中，有两个玩家，每个人都有自己的游戏块和一个 7×7 的正方形网格。在游戏开始时，第一个玩家将自己的棋子放在任何一个方格上，第二位玩家同样将自己的棋子放在任何一个可用的方格上。之后，玩家轮流移动他们的棋子，就像国际象棋中的皇后一样（垂直、水平或对角线任意数量的空心方块）。当棋子被移动时，先前占据的方格被封锁，并且不能用于游戏的剩余部分。第一个无法移动皇后的玩家判负。

在这种被称为 Impact Isolation 的变体中，每个玩家的棋子在向任何方向移动超过一个方块时都会在自身周围产生一个撞击坑。这个撞击坑阻挡了皇后移动到的位置周围的 4 个位置（北、南、东、西）。我们从这个变体中的 9×9 网格开始。两个玩家都不能继续或穿过被这种影响阻挡的方块。为清楚起见，请检查以下场景：

Q1 将他们的女王放在棋盘上，Q2 紧随其后。

<div>
<img src="./img/impact_1.png" width="500"/>
</div>

Q1 向右水平移动 4 个方块并在其周围产生影响，从而阻止 Q2 的一些潜在未来移动。Q1 的先前位置也将受到影响。

<div>
<img src="./img/impact_2.png" width="500"/>
</div>

Q2 的移动不超过一个方块，因此不会产生任何影响，而只是阻塞其先前所在的位置。

<div>
<img src="./img/impact_3.png" width="500"/>
</div>

Q1 向西南斜线移动。

<div>
<img src="./img/impact_4.png" width="500"/>
</div>

游戏将以类似的方式继续，直到任何一个皇后没有可移动的单元格。



### 为了更清楚地理解游戏规则，你可以尝试使用下面的互动工具与随机玩家或自己玩游戏

In [3]:
# 以下两行确保从 .py 脚本中导入任何内容
# 如果编辑和保存会自动重新加载（例如 local unit tests or players)
%load_ext autoreload
%autoreload 2
from board_viz import ReplayGame, InteractiveGame
from isolation import Board
from test_players import RandomPlayer

In [4]:
# 如果你想和自己玩，请将 RandomPlayer() 替换为 None
ig = InteractiveGame(RandomPlayer(), show_legal_moves=True)

GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001', height='auto', width='au…

move (4, 1) is illegal!
move (3, 1) is illegal!
Game is over, the winner is: Player - Q1
The game is over!
The game is over!


### 你可以做的另一件事是模拟两个玩家之间的游戏并重播。

**运行下一个单元格，在滑块正上方的文本输入框内单击，然后按向上或向下。**

In [5]:
# 这是一个可视化 2 名随机玩家的游戏重播的示例
game = Board(RandomPlayer(), RandomPlayer(), 9, 9)
winner, move_history, termination = game.play_isolation(time_limit=1000, print_moves=False)

bg = ReplayGame(game, move_history, show_legal_moves=True)
bg.show_board()

GridspecLayout(children=(GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001',…

---
<!-- Hi there! -->

# 重要文件

1. `isolation.py`: 包括 `Board` 类和用于将游戏打印为文本的函数。 
2. `notebook.ipynb`: 你将在其中为你的代理实现所需的方法。
3. `player_submission_tests.py`: 测试以验证你的代理。
4. `test_players.py`: 包含2种玩家类型供你测试代理：
    - `RandomPlayer` - 从可用的合法移动中随机选择
    - `HumanPlayer` - 你与AI对战 


---
<!-- Hi there! -->

# 任务

你将需要实现评估函数和游戏玩法。你的目标是实现以下部分：

1. 评估函数 ( `OpenMoveEvalFn` 或 `CustomEvalFn` )
2. 极小极大算法 ( `minimax` )
3. Alpha-beta 剪枝 ( `alphabeta` )
4. 根据你尝试处理的部分调整 `move()` 函数。

### 评估函数

这些函数将在你的 AI 选择动作时给出价值判断。有2个类：

- `OpenMoveEvalFn` - 返回值为你的 AI 玩家的合法移动数减去对手玩家的合法移动数。
- `CustomEvalFn` - 创建自己的评估函数。


### CustomPlayer

任务的核心。 关于这个类的一些注意事项:

- 可以更改提供的函数签名中的默认值。例如，当你创建自己的评估函数 `CustomEvalFn` 后，可以更改 `__init__` 中的默认值以使用新的评估函数。 
- 可以更改每个提供的函数的内容。例如，当你准备好使用 `alphabeta()` 时，您应该更新 `move()` 以改用该函数。
- 你可以自由地向类中添加更多方法。

你的 AI 代理每轮行动的时间有限（1 秒）。

如果您想提高基本性能，这里有一些建议：

- 使用分区技术。
- 存储过去动作的评估分数。
- 修改您的评估函数以说明必胜的移动。
- 优化经常调用的函数。
- 对节点进行排序以最大化修剪。

---
<!-- Hi there! -->

# 导入外部模块

In [6]:
# 以下两行确保从 .py 脚本中导入任何内容
# 如果编辑和保存会自动重新加载（例如 local unit tests or players)
%load_ext autoreload
%autoreload 2
import player_submission_tests as tests
from test_players import HumanPlayer, RandomPlayer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#export
import time
from isolation import Board

---
<!-- Hi there! -->

# 评估函数

## OpenMoveEvalFn
- 这是你编写评估函数以评估棋盘状态的地方。

以下是一些可能会对你实现 `OpenMoveEvalFn()` 有用的方法:

In [8]:
Board.get_player_moves??

In [9]:
Board.get_opponent_moves??

In [10]:
#export
class OpenMoveEvalFn:
    def score(self, game, my_player=None):
        """Score the current game state
        Evaluation function that outputs a score equal to how many
        moves are open for AI player on the board minus how many moves
        are open for Opponent's player on the board.

        Note:
            If you think of better evaluation function, do it in CustomEvalFn below.

            Args
                game (Board): The board and game state.
                my_player (Player object): This specifies which player you are.

            Returns:
                float: The current state's score. MyMoves-OppMoves.

            """
        player_move = game.get_player_moves(my_player)      # 我的合法移动数
        opponent_move = game.get_opponent_moves(my_player)  # 对手的合法移动数
        return len(player_move) - len(opponent_move)


######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.correctOpenEvalFn(OpenMoveEvalFn)
################ END OF LOCAL TEST CODE SECTION ######################


OpenMoveEvalFn Test: This board has a score of -11.



## CustomEvalFn
- 编辑下面的内容，以提出你自己改进的评估函数（可选）。
- 您需要在 CustomPlayer 中编辑 `eval_fn()` ，让你的 AI 代理在与测试代理对战时使用上述自定义评估函数。你可能还需要在 CustomPlayer 类中再次更改 `move()` 方法。

In [11]:
#export
class CustomEvalFn:
    def __init__(self):
        pass

    def score(self, game, my_player=None):
        """Score the current game state.
        
        Custom evaluation function that acts however you think it should. This
        is not required but highly encouraged if you want to build the best
        AI possible.
        
        Args:
            game (Board): The board and game state.
            my_player (Player object): This specifies which player you are.
            
        Returns:
            float: The current state's score, based on your own heuristic.
        """

        player_move = game.get_player_moves(my_player)      # 我的合法移动数
        opponent_move = game.get_opponent_moves(my_player)  # 对手的合法移动数
        
        if len(player_move) == 0:         # 检查到失败，赋予最低分数
            return float("-inf")
        if len(opponent_move) == 0:       # 检查到获胜，赋予最高分数
            return float("inf")
        
        '''以下加权为猜想，在合法步数之差相同时会有影响。
        测试时我方评估函数为CustomEvalFn，对手评估函数为OpenMoveEvalFn。
        (test.beatRandom增加一个参数eval_fn=None，不为None时作为我方的评估函数，
        将对手代理由 r = RandomPlayer() 修改为 r = yourAgent()，进行测试即可)，
        另外可以交换p和r的评估函数来测试，测试后手情况下的胜率。
        directions = [(-1, -1), (-1, 0), (-1, 1),
                      (0, -1), (0, 1),
                      (1, -1), (1, 0), (1, 1)]
        player_pos0, player_pos1 = game.get_player_position(my_player)
        opponent_pos0, opponent_pos1 = game.get_opponent_position(my_player)
        # 离中央的距离
        dist = abs((player_pos0 - game.height//2 + player_pos1 - game.width//2)/(game.height//2+game.width//2))
        # 计算我方周围空白格数加上对手周围隔离格数
        res = 8
        for direction in directions:
            col = direction[0] + player_pos0
            row = direction[1] + player_pos1
            if game.space_is_open(col, row):
                res += 1
            col = direction[0] + opponent_pos0
            row = direction[1] + opponent_pos1
            if game.space_is_open(col, row):
                res -= 1
        return len(player_move) - len(opponent_move) + res/32 + (1-dist)/2
        '''
        return len(player_move) - len(opponent_move) #+ len(player_move) / len(opponent_move) 
######################################################################
############ DON'T WRITE ANY CODE OUTSIDE THE CLASS! #################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################

---
<!-- Hi there! -->

# 游戏代理

## CustomPlayer
- CustomPlayer 是将用于玩 isolation 游戏的玩家对象。
- `move()` 方法将用于向你传递游戏板的当前状态。
- `move()` 方法的内容将由你更改。 虽然您可以使用迭代深化和 Alpha-Beta (ID+AB) 完成所有内容, 但这很容易出错，因此建议从 MiniMax (MM) 开始，然后实施 Alpha-Beta (AB)，然后再进行 ID+AB。


In [13]:
#export
class CustomPlayer:
    # TODO: finish this class!
    """Player that chooses a move using your evaluation function
    and a minimax algorithm with alpha-beta pruning.
    You must finish and test this player to make sure it properly
    uses minimax and alpha-beta to return a good move."""

    def __init__(self, search_depth=2, eval_fn=OpenMoveEvalFn()):
        """Initializes your player.
        
        if you find yourself with a superior eval function, update the default
        value of `eval_fn` to `CustomEvalFn()`
        
        Args:
            search_depth (int): The depth to which your agent will search
            eval_fn (function): Evaluation function used by your agent
        """
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """Called to determine one move by your agent

        Note:
            1. Do NOT change the name of this 'move' function. We are going to call
            this function directly.
            2. Call alphabeta instead of minimax once implemented.
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: (int,int): Your best move
        """
        if game.move_count == 0:
            return (0,1)
        '''
        player_pos0, player_pos1 = game.get_active_position()
        w, h = game.width, game.height
        pos = [(0,1),(1,0),(7,0),(8,1),(0,7),(1,8),(7,8),(8,7)]
        if game.get_active_player() is game.__player_1__:
            for p in pos:                    # 特殊位置
                if p in legal_moves: 
                    return p
            reflect_move = (h-player_pos0-1, w-player_pos1-1)
            if reflect_move in legal_moves:  # 镜像位置
                return reflect_move
        else:
            reflect_move = (h-player_pos0-1, w-player_pos1-1)
            if reflect_move in legal_moves: 
                return reflect_move
            for p in pos:
                if p in legal_moves: 
                    return p
        '''
            
        # minimax，默认迭代深度是2
        # best_move, utility = minimax(self, game, time_left, depth=self.search_depth)
        # alphabeta，默认迭代深度是6
        best_move = game.NOT_MOVED
        best_val = float('-inf')
        depth = 2
        while time_left() > 100:
            move, utility, true_depth = alphabeta(self, game, time_left, depth = depth)
            if utility == best_val:
                best_move = move
                break
            depth += 2
            if true_depth < 2:
                best_move = move
            else:
                break
        return best_move

    def utility(self, game, my_turn):
        """You can handle special cases here (e.g. endgame)"""
        if my_turn:
            return self.eval_fn.score(game, game.get_active_player())
        else:
            return self.eval_fn.score(game, game.get_inactive_player())

###################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE CLASS! ################
###### IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ###########
###################################################################

---
<!-- Hi there! -->

# Minimax
- 这是你实现 minimax 算法的地方。
- 实施 MM 后，有望在 **>=90% 的情况下击败随机玩家**。
- 有用的函数： 来自 `isolation.py` 的 `forecast_move()`方法和来自 OpenMoveEvalFn 的 `score()` 方法。

In [15]:
#export
def minimax(player, game, time_left, depth = 2, my_turn=True):
    """Implementation of the minimax algorithm.
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer()
            that represents your agent. It is used to call anything you
            need from the CustomPlayer class (the utility() method, for example,
            or any class variables that belong to CustomPlayer()).
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        my_turn (bool): True if you are computing scores during your turn.

    Returns:
        (tuple, int): best_move, val
    """

    time_cushion = 50 # 缓冲时间         
    
    # 搜索深度为0时返回当前的局部最优解
    if depth == 0: 
        return (game.NOT_MOVED, player.utility(game, my_turn))
    
    # 轮到我方时，选择评估值最大的移动
    if my_turn:
        best_val = float("-inf")
        best_move = game.NOT_MOVED
        for move in game.get_active_moves(): # 对我方所有合法移动进行轮询，找出评估值最大的移动
            # 我方行动后的新的棋盘状态(该函数会交换主动玩家和被动玩家，即child[0]的主动玩家是game的被动玩家)
            child = game.forecast_move(move)
            if time_left() < time_cushion:   # 剩余时间小于缓冲时间，停止搜索，执行当前最优解
                value = player.utility(game, True)
            else:
                value = minimax(player, child[0], time_left, depth - 1, False)[1]
            # 更新最大评估值及其移动
            if value > best_val:
                best_val = value
                best_move = move
    # 轮到对方时，选择评估值最小的节点
    else:
        best_val = float("inf")
        best_move = game.NOT_MOVED
        for move in game.get_active_moves(): # 此时对方是主动玩家，仍然选择函数get_active_moves
            child = game.forecast_move(move)
            if time_left() < time_cushion:
                value = player.utility(game, False)
            else:
                value = minimax(player, child[0], time_left, depth - 1, True)[1]
            if value < best_val:
                best_val = value
                best_move = move
    return best_move, best_val


######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.minimaxTest(CustomPlayer, minimax)  
################ END OF LOCAL TEST CODE SECTION ######################

Now running Minimax test 1.

Minimax passed for depth:  1
Minimax passed for depth:  2
Minimax passed for depth:  3
Minimax passed for depth:  4
Minimax passed for depth:  5

Now running Minimax test 2.

Minimax passed for depth:  1
Minimax passed for depth:  2
Minimax passed for depth:  3
Minimax passed for depth:  4
Minimax passed for depth:  5
Minimax Test: Runs Successfully!


---
<!-- Hi there! -->

# AlphaBeta
- 这是你实现 AlphaBeta 算法的地方。
- 实施 AB 后，有望在 **>=70% 的情况下击败使用 Minimax 算法(迭代深度为2)的玩家**。
- 有用的函数： 来自 `isolation.py` 的 `forecast_move()`方法和来自 OpenMoveEvalFn 的 `score()` 方法。

In [19]:
#export
# 排序函数
def evaluate(game, moves):
    move_score = {}
    directions = [(-1, -1), (-1, 0), (-1, 1),
                      (0, -1), (0, 1),
                      (1, -1), (1, 0), (1, 1)]
    for move in moves:
        res = 0
        for direction in directions:
            col = direction[0] + move[0]
            row = direction[1] + move[1]
            if game.space_is_open(col, row):
                res += 1
        move_score[move] = res
        
    return move_score

# 对节点进行排序以最大化修剪
def alphabeta(player, game, time_left, depth = 6, alpha=float("-inf"), beta=float("inf"), my_turn=True):
    """Implementation of the alphabeta algorithm.
    
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer() 
            that represents your agent. It is used to call anything you need 
            from the CustomPlayer class (the utility() method, for example, 
            or any class variables that belong to CustomPlayer())
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        alpha (float): Alpha value for pruning
        beta (float): Beta value for pruning
        my_turn (bool): True if you are computing scores during your turn.
        
    Returns:
        (tuple, int): best_move, val
    """
    
    time_cushion = 50 # 缓冲时间         
    
    # 搜索深度为0时返回当前的局部最优解
    if depth == 0: 
        return (game.NOT_MOVED, player.utility(game, my_turn), 0)
    
    true_depth = depth
    # 轮到我方时，选择评估值最大的移动
    if my_turn:
        best_val = alpha
        best_move = game.NOT_MOVED
        moves = game.get_active_moves()
        move_score = evaluate(game, moves)
        moves.sort(key = lambda x:move_score[x], reverse=False)
        for move in moves: # 对我方所有合法移动进行轮询，找出评估值最大的移动
            # 我方行动后的新的棋盘状态(该函数会交换主动玩家和被动玩家，即child[0]的主动玩家是game的被动玩家)
            child = game.forecast_move(move)
            if time_left() < time_cushion:   # 剩余时间小于缓冲时间，停止搜索，执行当前最优解
                value = player.utility(game, True)
            else:
                _, value, true_depth = alphabeta(player, child[0], time_left, depth - 1, alpha, beta, False)
            # 更新最大评估值及其移动
            if value > best_val:
                best_val = value
                best_move = move
            alpha = max(alpha, best_val) # 更新alpha，即max节点的下界
            if (beta <= alpha): # beta剪枝，当前节点的下界值已经大于beta(min父节点的下界)，当前节点剩下的子节点没有探索的意义
                break;
    # 轮到对方时，选择评估值最小的节点
    else:
        best_val = beta
        best_move = game.NOT_MOVED
        moves = game.get_active_moves()
        move_score = evaluate(game, moves)
        moves.sort(key = lambda x:move_score[x], reverse=True)
        for move in moves: # 此时对方是主动玩家，仍然选择函数get_active_moves
            child = game.forecast_move(move)
            if time_left() < time_cushion:
                value = player.utility(game, False)
            else:
                _, value, true_depth = alphabeta(player, child[0], time_left, depth - 1, alpha, beta, True)
            if value < best_val:
                best_val = value
                best_move = move
            beta = min(beta, best_val) # 更新beta，即min节点的上界
            if (beta <= alpha): # alpha剪枝，当前节点的上界值已经小于alpha(max父节点的下界)，当前节点剩下的子节点没有探索的意义
                break;
    return best_move, best_val, true_depth

# 未排序
def alphabeta2(player, game, time_left, depth = 5, alpha=float("-inf"), beta=float("inf"), my_turn=True):
    """Implementation of the alphabeta algorithm.
    
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer() 
            that represents your agent. It is used to call anything you need 
            from the CustomPlayer class (the utility() method, for example, 
            or any class variables that belong to CustomPlayer())
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        alpha (float): Alpha value for pruning
        beta (float): Beta value for pruning
        my_turn (bool): True if you are computing scores during your turn.
        
    Returns:
        (tuple, int): best_move, val
    """
    
    time_cushion = 50 # 缓冲时间         
    
    # 搜索深度为0时返回当前的局部最优解
    if depth == 0: 
        return (game.NOT_MOVED, player.utility(game, my_turn))
    
    # 轮到我方时，选择评估值最大的移动
    if my_turn:
        best_val = alpha
        best_move = game.NOT_MOVED
        for move in game.get_active_moves(): # 对我方所有合法移动进行轮询，找出评估值最大的移动
            # 我方行动后的新的棋盘状态(该函数会交换主动玩家和被动玩家，即child[0]的主动玩家是game的被动玩家)
            child = game.forecast_move(move)
            if time_left() < time_cushion:   # 剩余时间小于缓冲时间，停止搜索，执行当前最优解
                value = player.utility(game, True)
            else:
                value = alphabeta(player, child[0], time_left, depth - 1, alpha, beta, False)[1]
            # 更新最大评估值及其移动
            if value > best_val:
                best_val = value
                best_move = move
            alpha = max(alpha, best_val) # 更新alpha，即max节点的下界
            if (beta <= alpha): # beta剪枝，当前节点的下界值已经大于beta(min父节点的下界)，当前节点剩下的子节点没有探索的意义
                break;
    # 轮到对方时，选择评估值最小的节点
    else:
        best_val = beta
        best_move = game.NOT_MOVED
        for move in game.get_active_moves(): # 此时对方是主动玩家，仍然选择函数get_active_moves
            child = game.forecast_move(move)
            if time_left() < time_cushion:
                value = player.utility(game, False)
            else:
                value = alphabeta(player, child[0], time_left, depth - 1, alpha, beta, True)[1]
            if value < best_val:
                best_val = value
                best_move = move
            beta = min(beta, best_val) # 更新beta，即min节点的上界
            if (beta <= alpha): # alpha剪枝，当前节点的上界值已经小于alpha(max父节点的下界)，当前节点剩下的子节点没有探索的意义
                break;
    return best_move, best_val

######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
#tests.beatRandom(oppPlayer, CustomPlayer, CustomEvalFn())
tests.minimaxTest(CustomPlayer, alphabeta2)
################ END OF LOCAL TEST CODE SECTION ######################

Now running Minimax test 1.

Minimax passed for depth:  1
Minimax passed for depth:  2
Minimax passed for depth:  3
Minimax passed for depth:  4
Minimax passed for depth:  5

Now running Minimax test 2.

Minimax passed for depth:  1
Minimax passed for depth:  2
Minimax passed for depth:  3
Minimax passed for depth:  4
Minimax passed for depth:  5
Minimax Test: Runs Successfully!


---
<!-- Hi there! -->

# 提高性能
实现AB后，AI 代理的性能大大增强，但它依然不够强大，可以从以下方面进行改进：

- 使用迭代加深。任何使用相同算法进行更深入搜索的代理都可能具有更大的优势，我们可以在不耗尽时间的情况下找到最大搜索深度。每次从迭代深度为2开始，逐次增加迭代深度（每次加2），直到时间耗尽（参考 `alphabeta()` 中的true_depth，它会告诉 `move()` 是否能在规定时间内达到最大深度）。或许，你会指出前面的几次迭代浪费时间，但是对于最后较大的迭代深度而言，这点时间根本不值一提。如果幸运的话，能在迭代深度较小时找到最优的移动，此外，如果最后一次较深的迭代因时间不足而不幸中止，那么前面的迭代结果将作为移动的方案。

- 对节点进行排序以最大化修剪。我们希望在 beta/alpha 剪枝时，尽可能早的遇到评估值较 小/大 的节点，因此对节点进行排序非常有必要，这意味着我们需要在不探索该节点的情况下对其进行预评估，预评估的分数应该尽可能与评估函数的结果相关，参考 `evaluate()` 。

- 修改评估函数以说明必胜的移动。检查获胜的移动并确定它们的优先级，对于己方的获胜移动赋予无穷大，对方的获胜移动赋予无穷小。

- 考虑一些具有较大优势的位置。在己方先手的情况下，我们可以不进行迭代而直接给出一些具有较大优势的位置，这需要我们提前耗费较长的时间来进行迭代从而获得这些位置。

---

# 代理大战
现在，我们以及实现了一个强大的 AI 代理Q1，是时候与玩家Q2进行对决了。

attempts是测试的次数，默认情况下是10。

我们给出以下三类玩家，修改测试函数的第一个参数来进行选择:
    - `RandomPlayer` - 从可用的合法移动中随机选择
    - `HumanPlayer` - 你与AI对战
    - `oppPlayer` - 对手的代理，这里使用了与 AI 相同的代理。不同的是，其迭代算法 `alphabeta2()` 与 `alphabeta()` 稍有不同，它不对最后一次迭代               是否成功进行判断，直接接收最后一次迭代的结果。此外，它只会使用默认的评估函数，而你可以给 AI 代理使用自己的评估函数，如果               测试函数的第三个参数为空，AI 代理将使用默认的评估函数。

In [20]:
#export
# 对手的代理，用于测试
class oppPlayer:
    # TODO: finish this class!
    """Player that chooses a move using your evaluation function
    and a minimax algorithm with alpha-beta pruning.
    You must finish and test this player to make sure it properly
    uses minimax and alpha-beta to return a good move."""

    def __init__(self, search_depth=2, eval_fn=OpenMoveEvalFn()):
        """Initializes your player.
        
        if you find yourself with a superior eval function, update the default
        value of `eval_fn` to `CustomEvalFn()`
        
        Args:
            search_depth (int): The depth to which your agent will search
            eval_fn (function): Evaluation function used by your agent
        """
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """Called to determine one move by your agent

        Note:
            1. Do NOT change the name of this 'move' function. We are going to call
            this function directly.
            2. Call alphabeta instead of minimax once implemented.
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: (int,int): Your best move
        """
        #legal_moves = game.get_active_moves()
        # 占据中央位置
        if game.move_count == 1: 
            return (4,4)
        '''
        player_pos0, player_pos1 = game.get_active_position()
        w, h = game.width, game.height
        pos = [(0,1),(1,0),(7,0),(8,1),(0,7),(1,8),(7,8),(8,7)]
        if game.get_active_player() is game.__player_1__:
            for p in pos:                    # 特殊位置
                if p in legal_moves: 
                    return p
            reflect_move = (h-player_pos0-1, w-player_pos1-1)
            if reflect_move in legal_moves:  # 镜像位置
                return reflect_move
        else:
            reflect_move = (h-player_pos0-1, w-player_pos1-1)
            if reflect_move in legal_moves: 
                return reflect_move
            for p in pos:
                if p in legal_moves: 
                    return p
        '''
            
        # minimax，默认迭代深度是2
        #best_move, utility = minimax(self, game, time_left, depth=self.search_depth)
        # alphabeta，默认迭代深度是6
        best_move = game.NOT_MOVED
        depth = 2
        while time_left() > 100:
            move, utility, true_depth = alphabeta(self, game, time_left, depth = depth)
            if utility == float('inf'):
                best_move = move
                break
            depth += 2
            if true_depth < 2:
                best_move = move
            else:
                break
        return best_move

    def utility(self, game, my_turn):
        """You can handle special cases here (e.g. endgame)"""
        if my_turn:
            return self.eval_fn.score(game, game.get_active_player())
        else:
            return self.eval_fn.score(game, game.get_inactive_player())

######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
attempts = 10
tests.beatRandom(oppPlayer, CustomPlayer, attempts, CustomEvalFn())
################ END OF LOCAL TEST CODE SECTION ######################



 CustomPlayer - Q1  Turn
move chosen:  (0, 1)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |Q1|  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (4, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |Q1|  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |Q2|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (1, 0)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |Q1|  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |Q2|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |

move chosen:  (8, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |Q1|  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |><|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |><|  |  |  |  |
8 |  |  |  |><|Q2|><|  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (7, 2)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |><|  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |><|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |><|  |  |  |  |  |  |
7 |  |><|Q1|><|><|  |  |  |  |
8 |  |  |><|><|Q2|><|  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (7, 5)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |><|  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |><|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |><|  |  |  |  |  |  |
7 |  |><|Q1|><|><|Q2

move chosen:  (1, 0)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |Q1|  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |Q2|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (1, 1)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |Q1|Q2|><|  |  |  |  |  |  |
2 |  |><|  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |><|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (8, 0)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |><|Q2|><|  |  |  |  |  |  |
2 |  |><|  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |><|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |><|  |  |  |  |  

move chosen:  (1, 0)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |Q1|  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |Q2|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (8, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |Q1|  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |><|  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |><|  |  |  |  |
8 |  |  |  |><|Q2|><|  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (5, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |><|  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |><|  |  |  |  |
5 |  |  |  |><|Q1|><|  |  |  |
6 |  |  |  |  |><|  |  |  |  |
7 |  |  |  |  |><|  

move chosen:  (2, 3)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|><|><|  |  |  |  |  |
1 |><|  |><|Q1|  |  |  |  |  |
2 |  |  |><|Q2|><|  |  |  |  |
3 |><|><|  |><|  |  |  |  |  |
4 |><|><|><|  |><|  |  |  |><|
5 |><|><|  |  |  |  |  |><|><|
6 |><|><|  |  |  |  |  |><|><|
7 |><|  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (1, 5)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|><|><|  |><|  |  |  |
1 |><|  |><|><|><|Q1|><|  |  |
2 |  |  |><|Q2|><|><|  |  |  |
3 |><|><|  |><|  |  |  |  |  |
4 |><|><|><|  |><|  |  |  |><|
5 |><|><|  |  |  |  |  |><|><|
6 |><|><|  |  |  |  |  |><|><|
7 |><|  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (4, 5)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|><|><|  |><|  |  |  |
1 |><|  |><|><|><|Q1|><|  |  |
2 |  |  |><|><|><|><|  |  |  |
3 |><|><|  |><|  |><|  |  |  |
4 |><|><|><|  |><|Q2|><|  |><|
5 |><|><|  |  |  |><|  |><|><|
6 |><|><|  |  |  |  |  |><|><|
7 |><|  |  |  |  |  

...

move chosen:  (0, 5)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |><|Q1|><|><|><|
1 |><|  |  |  |  |><|><|><|  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |><|><|><|><|  |  |
5 |  |  |  |><|><|><|><|><|  |
6 |  |  |  |  |><|><|><|><|Q2|
7 |  |  |  |  |><|><|><|  |  |
8 |  |  |  |><|><|><|  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (5, 8)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |><|Q1|><|><|><|
1 |><|  |  |  |  |><|><|><|  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |><|><|><|><|  |  |
5 |  |  |  |><|><|><|><|><|Q2|
6 |  |  |  |  |><|><|><|><|><|
7 |  |  |  |  |><|><|><|  |  |
8 |  |  |  |><|><|><|  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (1, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |><|><|><|><|><|
1 |><|  |  |  |Q1|><|><|><|  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |><|><|><|><|  |  |
5 |  |  |  |><|><|><|><|><|Q2|
6 |  |  |  |  |><|><|><|><|><|
7 |  |  |  |  |><|><

move chosen:  (2, 7)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |><|><|  |  |  |  |  |
2 |  |  |  |><|><|  |  |Q1|><|
3 |><|><|><|  |><|  |  |><|><|
4 |><|><|><|><|><|  |  |  |><|
5 |><|><|><|><|Q2|  |  |  |  |
6 |  |  |><|  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (4, 5)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |><|><|  |  |  |  |  |
2 |  |  |  |><|><|  |  |Q1|><|
3 |><|><|><|  |><|  |  |><|><|
4 |><|><|><|><|><|Q2|  |  |><|
5 |><|><|><|><|><|  |  |  |  |
6 |  |  |><|  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (3, 6)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |><|><|  |  |  |  |  |
2 |  |  |  |><|><|  |  |><|><|
3 |><|><|><|  |><|  |Q1|><|><|
4 |><|><|><|><|><|Q2|  |  |><|
5 |><|><|><|><|><|  |  |  |  |
6 |  |  |><|  |  |  |  |  |  |
7 |  |  |  |  |  |  

move chosen:  (3, 8)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |><|  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |><|
3 |  |  |  |  |Q1|  |  |><|Q2|
4 |  |  |  |><|><|  |  |  |><|
5 |  |  |  |><|><|><|  |  |  |
6 |  |  |  |  |><|><|  |  |  |
7 |  |  |  |  |><|><|  |  |  |
8 |  |  |  |><|><|><|  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (0, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |><|Q1|><|  |  |  |
1 |><|  |  |  |><|  |  |  |  |
2 |  |  |  |  |  |  |  |  |><|
3 |  |  |  |  |><|  |  |><|Q2|
4 |  |  |  |><|><|  |  |  |><|
5 |  |  |  |><|><|><|  |  |  |
6 |  |  |  |  |><|><|  |  |  |
7 |  |  |  |  |><|><|  |  |  |
8 |  |  |  |><|><|><|  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (5, 6)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |><|Q1|><|  |  |  |
1 |><|  |  |  |><|  |  |  |  |
2 |  |  |  |  |  |  |  |  |><|
3 |  |  |  |  |><|  |  |><|><|
4 |  |  |  |><|><|  |><|  |><|
5 |  |  |  |><|><|><|Q2|><|  |
6 |  |  |  |  |><|><|><|  |  |
7 |  |  |  |  |><|><

move chosen:  (3, 7)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |><|  |  |  |  |  |  |
2 |  |  |  |Q1|  |  |  |  |  |
3 |  |  |  |  |  |  |  |Q2|><|
4 |  |  |  |  |><|  |  |><|><|
5 |  |  |  |  |  |  |  |  |><|
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (3, 3)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |><|  |  |  |  |  |  |
2 |  |  |  |><|  |  |  |  |  |
3 |  |  |  |Q1|  |  |  |Q2|><|
4 |  |  |  |  |><|  |  |><|><|
5 |  |  |  |  |  |  |  |  |><|
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (3, 6)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|  |  |  |  |  |  |  |
1 |  |  |><|  |  |  |  |  |  |
2 |  |  |  |><|  |  |  |  |  |
3 |  |  |  |Q1|  |  |Q2|><|><|
4 |  |  |  |  |><|  |  |><|><|
5 |  |  |  |  |  |  |  |  |><|
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  

move chosen:  (7, 2)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |><|><|><|  |><|><|><|><|
1 |  |  |><|><|  |  |><|><|  |
2 |  |  |Q1|><|  |  |  |  |  |
3 |><|><|><|><|  |  |><|><|><|
4 |><|><|><|><|><|  |  |><|><|
5 |><|><|><|><|><|><|  |  |><|
6 |><|><|><|><|><|  |  |  |  |
7 |><|><|Q2|  |  |  |  |  |  |
8 |><|><|  |  |  |  |  |  |  |

 CustomPlayer - Q1  Turn

 oppPlayer - Q2  has won. Reason:  CustomPlayer - Q1 made an illegal move.

 CustomPlayer - Q1  Turn
move chosen:  (0, 1)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |Q1|  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 oppPlayer - Q2  Turn
move chosen:  (4, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |Q1|  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  

---